<a href="https://colab.research.google.com/github/fboldt/aulas-am-bsi/blob/main/aula09a_decision_tree_for_discrete_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

car_evaluation = fetch_ucirepo(id=19)

X = car_evaluation.data.features.to_numpy()
y = car_evaluation.data.targets.to_numpy()[:,0]

print(car_evaluation.variables)

       name     role         type demographic  \
0    buying  Feature  Categorical        None   
1     maint  Feature  Categorical        None   
2     doors  Feature  Categorical        None   
3   persons  Feature  Categorical        None   
4  lug_boot  Feature  Categorical        None   
5    safety  Feature  Categorical        None   
6     class   Target  Categorical        None   

                                         description units missing_values  
0                                       buying price  None             no  
1                           price of the maintenance  None             no  
2                                    number of doors  None             no  
3              capacity in terms of persons to carry  None             no  
4                           the size of luggage boot  None             no  
5                        estimated safety of the car  None             no  
6  evaulation level (unacceptable, acceptable, go...  None             no  

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
set(y)

{'acc', 'good', 'unacc', 'vgood'}

In [6]:
len(y)

1728

In [7]:
combinations = 1
for i in range(X.shape[1]):
  values = set(X[:,i])
  combinations *= len(values)
  print(f"{i}: {car_evaluation.variables['name'][i]}:\t{values}")
print(combinations)

0: buying:	{'vhigh', 'med', 'high', 'low'}
1: maint:	{'vhigh', 'med', 'high', 'low'}
2: doors:	{'4', '3', '5more', '2'}
3: persons:	{'more', '4', '2'}
4: lug_boot:	{'small', 'med', 'big'}
5: safety:	{'high', 'med', 'low'}
1728


In [12]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from collections import Counter

def most_common(lst):
  data = Counter(lst)
  return data.most_common(1)[0][0]

class ZeroR(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
      self.answer = most_common(y)
      return self
    def predict(self, X):
      return [self.answer] * len(X)

model = ZeroR()
model.fit(X, y)
ypred = model.predict(X)
print(accuracy_score(y, ypred))

0.7002314814814815


In [13]:
model.answer

'unacc'

In [14]:
model = ZeroR()
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.6994219653179191


In [55]:
import numpy as np

# apenas uma característica
class DecisionTree(BaseEstimator, ClassifierMixin):
  def __init__(self, feature=0):
    self.feature = feature

  def fit(self, X, y):
    self.value = list(set(X[:,self.feature]))[0]
    equals = X[:,self.feature] == self.value
    if sum(equals)>0 and sum(~equals)>0:
      self.equals = DecisionTree(self.feature).fit(X[equals], y[equals])
      self.not_equals = DecisionTree(self.feature).fit(X[~equals], y[~equals])
    else:
      self.answer = most_common(y[equals])
      self.n_samples = len(y)
    return self

  def predict(self, X):
    if hasattr(self, 'answer'):
      return [self.answer] * len(X)
    else:
      equals = X[:,self.feature] == self.value
    return np.where(equals, self.equals.predict(X), self.not_equals.predict(X))

model = DecisionTree()
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.6994219653179191


In [56]:
model = DecisionTree(1)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.6994219653179191


In [57]:
def print_tree(tree, depth=0):
  if hasattr(tree, 'answer'):
    print('\t' * depth, f"answer: {tree.answer} ({tree.n_samples})")
  else:
    print('\t' * depth, f"value: {tree.value}")
    print_tree(tree.equals, depth+1)
    print_tree(tree.not_equals, depth+1)

print_tree(model)

 value: high
	 answer: unacc (343)
	 value: vhigh
		 answer: unacc (327)
		 value: med
			 answer: unacc (356)
			 answer: unacc (356)


In [27]:
def gini(y):
  label = np.unique(y)
  aux = 0
  for l in label:
    p = np.sum(y==l)/len(y)
    aux += p**2
  return 1-aux

print(gini(y_train))

0.4570454112310227


In [28]:
print(gini(np.ones(10)))

0.0


In [35]:
def impurity_value(y, x, value, impurity_function=gini):
  equals = x == value
  imp_equals = impurity_function(y[equals])
  imp_not_equals = impurity_function(y[~equals])
  return len(y[equals])/len(y)*imp_equals + len(y[~equals])/len(y)*imp_not_equals

print(impurity_value(y_train, X_train[:,0], 'vhigh'))

0.4482135562918824


In [36]:
def best_split(y, x, impurity_function=gini):
  best_value = None
  best_impurity = 1
  for value in set(x):
    impurity = impurity_value(y, x, value, impurity_function)
    if impurity < best_impurity:
      best_impurity = impurity
      best_value = value
  return best_value, best_impurity

print(best_split(y_train, X_train[:,0]))

('vhigh', np.float64(0.4482135562918824))


In [38]:
def best_feature(y, X, impurity_function=gini):
  best_feature = None
  best_value = None
  best_impurity = 1
  for feature in range(X.shape[1]):
    value, impurity = best_split(y, X[:,feature], impurity_function)
    if impurity < best_impurity:
      best_impurity = impurity
      best_feature = feature
      best_value = value
  return best_feature, best_value, best_impurity

print(best_feature(y_train, X_train))

(5, 'low', np.float64(0.38499511557696947))


In [58]:
class DecisionTree(BaseEstimator, ClassifierMixin):
  def fit(self, X, y):
    self.feature, self.value, self.impurity = best_feature(y, X)
    equals = X[:,self.feature] == self.value
    if sum(equals)>0 and sum(~equals)>0:
      self.equals = DecisionTree().fit(X[equals], y[equals])
      self.not_equals = DecisionTree().fit(X[~equals], y[~equals])
    else:
      self.answer = most_common(y[equals])
      self.n_samples = len(y)
    return self

  def predict(self, X):
    if hasattr(self, 'answer'):
      return [self.answer] * len(X)
    else:
      equals = X[:,self.feature] == self.value
    return np.where(equals, self.equals.predict(X), self.not_equals.predict(X))

model = DecisionTree()
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.9739884393063584


In [59]:
print_tree(model)

 value: low
	 value: high
		 answer: unacc (123)
		 value: vhigh
			 answer: unacc (124)
			 value: med
				 answer: unacc (106)
				 answer: unacc (112)
	 value: 2
		 value: vhigh
			 answer: unacc (78)
			 value: high
				 answer: unacc (79)
				 value: med
					 answer: unacc (71)
					 answer: unacc (74)
		 value: vhigh
			 value: med
				 value: small
					 value: high
						 value: 2
							 value: more
								 answer: unacc (1)
								 answer: acc (1)
							 answer: acc (5)
						 answer: unacc (6)
					 value: med
						 value: 2
							 value: more
								 answer: acc (1)
								 answer: unacc (1)
							 value: 3
								 value: more
									 answer: acc (1)
									 value: high
										 answer: acc (1)
										 answer: unacc (1)
								 answer: acc (8)
						 answer: acc (14)
				 value: low
					 value: high
						 value: 2
							 value: small
								 value: more
									 answer: unacc (1)
									 answer: acc (1)
								 answer: acc (4)
							 answer: acc

In [60]:
from sklearn.model_selection import cross_validate, KFold

scores = cross_validate(DecisionTree(), X, y, cv=KFold(shuffle=True))
print(scores['test_score'])
print(scores['test_score'].mean())

[0.9566474  0.97109827 0.9566474  0.94492754 0.97681159]
0.9612264388037195


In [63]:
class DecisionTree(BaseEstimator, ClassifierMixin):
  def __init__(self, min_samples_leaf=1):
    self.min_samples_leaf = min_samples_leaf

  def fit(self, X, y):
    self.feature, self.value, self.impurity = best_feature(y, X)
    equals = X[:,self.feature] == self.value
    if sum(equals) >= self.min_samples_leaf and \
        sum(~equals) >= self.min_samples_leaf:
      self.equals = DecisionTree(min_samples_leaf=self.min_samples_leaf).fit(X[equals], y[equals])
      self.not_equals = DecisionTree(min_samples_leaf=self.min_samples_leaf).fit(X[~equals], y[~equals])
    else:
      self.answer = most_common(y[equals])
      self.n_samples = len(y)
    return self

  def predict(self, X):
    if hasattr(self, 'answer'):
      return [self.answer] * len(X)
    else:
      equals = X[:,self.feature] == self.value
    return np.where(equals, self.equals.predict(X), self.not_equals.predict(X))

model = DecisionTree(20)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.869942196531792


In [64]:
print_tree(model)

 value: low
	 value: high
		 answer: unacc (123)
		 value: vhigh
			 answer: unacc (124)
			 value: med
				 answer: unacc (106)
				 answer: unacc (112)
	 value: 2
		 value: vhigh
			 answer: unacc (78)
			 value: high
				 answer: unacc (79)
				 value: med
					 answer: unacc (71)
					 answer: unacc (74)
		 value: vhigh
			 value: med
				 answer: unacc (40)
				 value: low
					 answer: acc (37)
					 answer: unacc (78)
			 value: high
				 value: vhigh
					 answer: unacc (35)
					 value: small
						 value: high
							 answer: acc (20)
							 answer: unacc (21)
						 value: 2
							 answer: acc (23)
							 value: 3
								 answer: acc (20)
								 answer: acc (38)
				 value: low
					 value: high
						 answer: vgood (38)
						 answer: acc (38)
					 value: med
						 value: high
							 answer: good (42)
							 answer: acc (36)
						 value: small
							 value: high
								 answer: unacc (26)
								 answer: unacc (24)
							 value: med
								 answer: acc (51)


In [73]:
class DecisionTree(BaseEstimator, ClassifierMixin):
  def __init__(self, min_samples_leaf=1, max_depth=10000):
    self.min_samples_leaf = min_samples_leaf
    self.max_depth = max_depth

  def fit(self, X, y):
    self.feature, self.value, self.impurity = best_feature(y, X)
    equals = X[:,self.feature] == self.value
    if sum(equals) >= self.min_samples_leaf and \
        sum(~equals) >= self.min_samples_leaf and \
        self.max_depth > 0:
      self.equals = DecisionTree(min_samples_leaf=self.min_samples_leaf, \
                                 max_depth=self.max_depth-1).fit(X[equals], y[equals])
      self.not_equals = DecisionTree(min_samples_leaf=self.min_samples_leaf, \
                                 max_depth=self.max_depth-1).fit(X[~equals], y[~equals])
    else:
      self.answer = most_common(y[equals])
      self.n_samples = len(y)
    return self

  def predict(self, X):
    if hasattr(self, 'answer'):
      return [self.answer] * len(X)
    else:
      equals = X[:,self.feature] == self.value
    return np.where(equals, self.equals.predict(X), self.not_equals.predict(X))

model = DecisionTree(2, 6)
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.8728323699421965


In [74]:
print_tree(model)

 value: low
	 value: high
		 answer: unacc (123)
		 value: vhigh
			 answer: unacc (124)
			 value: med
				 answer: unacc (106)
				 answer: unacc (112)
	 value: 2
		 value: vhigh
			 answer: unacc (78)
			 value: high
				 answer: unacc (79)
				 value: med
					 answer: unacc (71)
					 answer: unacc (74)
		 value: vhigh
			 value: med
				 value: small
					 value: high
						 answer: acc (7)
						 answer: unacc (6)
					 value: med
						 answer: acc (13)
						 answer: acc (14)
				 value: low
					 value: high
						 answer: acc (19)
						 answer: acc (18)
					 answer: unacc (78)
			 value: high
				 value: vhigh
					 answer: unacc (35)
					 value: small
						 answer: acc (41)
						 answer: acc (81)
				 value: low
					 value: high
						 answer: vgood (38)
						 answer: acc (38)
					 value: med
						 answer: vgood (78)
						 answer: acc (149)


In [75]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 4.9 MB/s eta 0:00:00


In [78]:
import optuna
from sklearn.model_selection import cross_val_score

def objective(trial):
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 50)
    max_depth = trial.suggest_int('max_depth', 1, 30)

    model = DecisionTree(min_samples_leaf=min_samples_leaf, max_depth=max_depth)

    # Use cross-validation to evaluate the model
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best trial:")
print("  Value: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print("    {}: {}".format(key, value))

[I 2025-09-18 15:47:43,000] A new study created in memory with name: no-name-c277ebfb-0cb6-48b4-88d7-580dc11c00d3
[I 2025-09-18 15:47:43,465] Trial 0 finished with value: 0.8053785381677392 and parameters: {'min_samples_leaf': 20, 'max_depth': 28}. Best is trial 0 with value: 0.8053785381677392.
[I 2025-09-18 15:47:43,903] Trial 1 finished with value: 0.8140611102391043 and parameters: {'min_samples_leaf': 19, 'max_depth': 10}. Best is trial 1 with value: 0.8140611102391043.
[I 2025-09-18 15:47:44,555] Trial 2 finished with value: 0.9095510908805526 and parameters: {'min_samples_leaf': 5, 'max_depth': 22}. Best is trial 2 with value: 0.9095510908805526.
[I 2025-09-18 15:47:45,044] Trial 3 finished with value: 0.7228927954795166 and parameters: {'min_samples_leaf': 45, 'max_depth': 18}. Best is trial 2 with value: 0.9095510908805526.
[I 2025-09-18 15:47:45,833] Trial 4 finished with value: 0.8126118348767853 and parameters: {'min_samples_leaf': 21, 'max_depth': 27}. Best is trial 2 with

Best trial:
  Value:  0.9623606969078636
  Params: 
    min_samples_leaf: 1
    max_depth: 22


In [79]:
model = DecisionTree(min_samples_leaf=1, max_depth=22)

scores = cross_validate(model, X, y, cv=KFold(shuffle=True))
print(scores['test_score'])
print(scores['test_score'].mean())

[0.97687861 0.96531792 0.97398844 0.95942029 0.95072464]
0.9652659797268995
